## Maciej Olszewski

This is a jupyter notebook wrapper to run detect-classify traffic signs app **using cloud**.

_Google Cloud Platform_ version.

In [8]:
import os
import cv2
import json
import requests

import matplotlib.pyplot as plt
import numpy as np

from detect import Detector, NoDetectionsException

In [9]:
from datetime import datetime
# DIR_TEST_IMAGES = "/Users/machofv/Downloads/GTSDB-archive-2/TrainIJCNN2013/TrainIJCNN2013"
DIR_TEST_IMAGES = "/Users/machofv/Downloads/GTSDB-archive-2/TestIJCNN2013/TestIJCNN2013Download"

DIR_RESULTS = f"{os.path.abspath('')}/../results/{datetime.today()}"

HTTP_FUNCTION = "https://europe-central2-aesthetic-fiber-428811-m0.cloudfunctions.net/rs-recognize"

In [10]:
def __img_to_json(img: np.ndarray) -> dict:
    """Jsonifies given array."""
    # return json.dumps(img.tolist())
    return {"ndarray_data": json.dumps(img.tolist())}

def __send_request(message: dict, url: str):
    response = requests.post(url, json=message)
    if response.status_code == 200:
        # print("response:", response.text)

        response_dict = json.loads(response.text)

        return response_dict.get("prediction", "")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        raise Exception(f"Error: {response.status_code}, {response.text}")

In [11]:
def enhance_contrast_adaptive(img, clip_limit=2.0, tile_grid_size=(8, 8)):
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    img_yuv[:, :, 0] = clahe.apply(img_yuv[:, :, 0])  # Apply CLAHE to the Y (luminance) channel
    img_enhanced = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    return img_enhanced

def visualize(img, imgs_detected, predictions, j_counter, is_saved=False):
    plt.subplots_adjust(top=0.90)
    fig, axs = plt.subplots(1, 1+len(predictions), figsize=(15, 5))

    axs[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[0].set_title('Original image')
    axs[0].axis('off')

    for i, img_disp in enumerate(imgs_detected):
        axs[i+1].imshow(cv2.cvtColor(img_disp, cv2.COLOR_BGR2RGB))
        axs[i+1].set_title(predictions[i])
        axs[i+1].axis("off")
    plt.tight_layout()
    if is_saved:
        plt.savefig(f"{DIR_RESULTS}/result_{j_counter}")
    else:
        plt.show()

def visualize_enhanced(img, imgs_detected, imgs_enhanced, predictions, j_counter, is_saved=False):
    plt.subplots_adjust(top=0.90)
    fig, axs = plt.subplots(2, 1+len(predictions), figsize=(15, 5))

    axs[0][0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axs[0][0].set_title('Original image')
    axs[0][0].axis('off')
    axs[1][0].axis("off")

    for i, img_disp in enumerate(imgs_detected):
        axs[0][i+1].imshow(cv2.cvtColor(img_disp, cv2.COLOR_BGR2RGB))
        axs[0][i+1].set_title(predictions[i])
        axs[0][i+1].axis("off")

        axs[1][i+1].imshow(cv2.cvtColor(imgs_enhanced[i], cv2.COLOR_BGR2RGB))
        axs[1][i+1].set_title("(inc. contrast)")
        axs[1][i+1].axis("off")

    plt.tight_layout()
    if is_saved:
        plt.savefig(f"{DIR_RESULTS}/result_{j_counter}")
    else:
        plt.show()

In [12]:
def run_from_folder(DIR_IMAGES, detector, URL, is_enhanced=True, is_saved=False):
    test_imgs = os.listdir(DIR_IMAGES)
    for j, IMG_PATH in enumerate(test_imgs):

        imgs_detected = []
        if IMG_PATH == ".DS_Store":
            continue
        try:
            imgs_detected = detector.detect_yolo(f"{DIR_IMAGES}/{IMG_PATH}")
        except NoDetectionsException:
            continue
        except IOError:
            print(f"WARNING! - '{IMG_PATH}' file cannot be read! Skipping...")
            continue

        predictions = []
        if is_enhanced: imgs_enhanced = []

        if imgs_detected:
            for img_detected in imgs_detected:

                if is_enhanced:
                    # img_enhanced = cv2.convertScaleAbs(img_detected, alpha=1.5, beta=15)
                    img_enhanced = enhance_contrast_adaptive(img_detected)
                    imgs_enhanced.append(img_enhanced)
                    img_resized = np.expand_dims(cv2.resize(img_enhanced, (64, 64)), axis=0)
                    message_json = __img_to_json(img_resized)

                else:
                    img_resized = np.expand_dims(cv2.resize(img_detected, (64, 64)), axis=0)
                    message_json = __img_to_json(img_resized)                
                
                response = __send_request(message_json, URL)
                predictions.append(response)

                
            NUM_OF_PREDS = len(predictions)
            img = cv2.imread(f"{DIR_IMAGES}/{IMG_PATH}")

            # Visualize the data
            if is_enhanced:
                visualize_enhanced(img, imgs_detected, imgs_enhanced, predictions, j, is_saved=is_saved)
            else:
                visualize(img, imgs_detected, predictions, j, is_saved=is_saved)

In [13]:
detector = Detector()

In [14]:
os.makedirs(DIR_RESULTS, exist_ok=True)

run_from_folder(DIR_TEST_IMAGES, detector, HTTP_FUNCTION, is_enhanced=True, is_saved=True)


0: 384x640 1 Traffic sign, 16.9ms
Speed: 2.1ms preprocess, 16.9ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.9ms
Speed: 2.4ms preprocess, 24.9ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Traffic sign, 21.9ms
Speed: 2.0ms preprocess, 21.9ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Traffic signs, 21.6ms
Speed: 1.9ms preprocess, 21.6ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 Traffic signs, 18.3ms
Speed: 1.8ms preprocess, 18.3ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18.6ms
Speed: 2.1ms preprocess, 18.6ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Traffic signs, 18.7ms
Speed: 2.2ms preprocess, 18.7ms inference, 65.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Traffic signs, 24.1ms
Speed: 2.1ms preprocess, 24.1ms in

/var/folders/v3/vkc53k7s67jd8p8llsp_2czh0000gn/T/ipykernel_66708/1824003793.py:28: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(2, 1+len(predictions), figsize=(15, 5))



0: 384x640 (no detections), 19.3ms
Speed: 2.0ms preprocess, 19.3ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Traffic signs, 17.6ms
Speed: 1.8ms preprocess, 17.6ms inference, 47.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Traffic signs, 26.4ms
Speed: 1.9ms preprocess, 26.4ms inference, 11.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Traffic signs, 21.4ms
Speed: 1.8ms preprocess, 21.4ms inference, 48.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Traffic signs, 17.9ms
Speed: 2.0ms preprocess, 17.9ms inference, 11.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Traffic signs, 17.8ms
Speed: 1.8ms preprocess, 17.8ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Traffic signs, 19.1ms
Speed: 1.9ms preprocess, 19.1ms inference, 7.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Traffic signs, 18.2ms
Speed: 1.7ms preprocess, 18.2